In [1]:
1 + 1

2

In [2]:
str = 'stressed'
print(str[::-1])

desserts


"Hi He Lied Because Boron Could Not Oxidize Fluorine. New Nations Might Also Sign Peace Security Clause. Arthur King Can."という文を単語に分解し，1, 5, 6, 7, 8, 9, 15, 16, 19番目の単語は先頭の1文字，それ以外の単語は先頭に2文字を取り出し，取り出した文字列から単語の位置（先頭から何番目の単語か）への連想配列（辞書型もしくはマップ型）を作成せよ．

In [1]:
s = "Hi He Lied Because Boron Could Not Oxidize Fluorine. New Nations Might Also Sign Peace Security Clause. Arthur King Can."

In [7]:
l = s.split(" ")
one = [1, 5, 6, 7, 8, 9, 15, 16, 19]
m = {}
for i  in range(len(l)):
    text = ''
    if i+1 in one:
        text = l[i][0]
    else:
        text = l[i][:2]
    m[text] = i+1

m    

{'H': 1,
 'He': 2,
 'Li': 3,
 'Be': 4,
 'B': 5,
 'C': 6,
 'N': 7,
 'O': 8,
 'F': 9,
 'Ne': 10,
 'Na': 11,
 'Mi': 12,
 'Al': 13,
 'Si': 14,
 'P': 15,
 'S': 16,
 'Cl': 17,
 'Ar': 18,
 'K': 19,
 'Ca': 20}

In [5]:
f = open('hightemp.txt')

In [4]:
f.readlines()

[]

In [6]:
len(f.readlines())

24

In [157]:
with open('hightemp.txt', "r") as f:
    data = f.readlines() #データを読み込んでリストにする

col0 = []

for l in data:
    l = l.split('\t')
    col0.append(l[0])

In [158]:
col0

['高知県',
 '埼玉県',
 '岐阜県',
 '山形県',
 '山梨県',
 '和歌山県',
 '静岡県',
 '山梨県',
 '埼玉県',
 '群馬県',
 '群馬県',
 '愛知県',
 '千葉県',
 '静岡県',
 '愛媛県',
 '山形県',
 '岐阜県',
 '群馬県',
 '千葉県',
 '埼玉県',
 '大阪府',
 '山梨県',
 '山形県',
 '愛知県']

In [161]:
import collections
collections.Counter(col0)

Counter({'高知県': 1,
         '埼玉県': 3,
         '岐阜県': 2,
         '山形県': 3,
         '山梨県': 3,
         '和歌山県': 1,
         '静岡県': 2,
         '群馬県': 3,
         '愛知県': 2,
         '千葉県': 2,
         '愛媛県': 1,
         '大阪府': 1})

In [178]:
sorted(collections.Counter(col0).items(), key=lambda x: x[1])

[('高知県', 1),
 ('和歌山県', 1),
 ('愛媛県', 1),
 ('大阪府', 1),
 ('岐阜県', 2),
 ('静岡県', 2),
 ('愛知県', 2),
 ('千葉県', 2),
 ('埼玉県', 3),
 ('山形県', 3),
 ('山梨県', 3),
 ('群馬県', 3)]

In [172]:
dict(collections.Counter(col0))

{'高知県': 1,
 '埼玉県': 3,
 '岐阜県': 2,
 '山形県': 3,
 '山梨県': 3,
 '和歌山県': 1,
 '静岡県': 2,
 '群馬県': 3,
 '愛知県': 2,
 '千葉県': 2,
 '愛媛県': 1,
 '大阪府': 1}